Not even a useful baseline. But will work to evaluate the model. Last competition's public NB that uses combined submission also has a binary output, and a score of 0.97. We'll use those probability scores with weights for each comment type to generate a final regressor score. Bertweet model will get preprocessed inputs (except url and twitter handle preprocessing) and a regressor head on top of its model is finetuned. 


# Test from some prediction
## Score generation

In [ ]:
import pandas as pd, gc, wt_text_processing_utils as wtp_utils, numpy as np
from tqdm import tqdm
tqdm.pandas()

dftest = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip")
dfpred = pd.read_csv("../input/classifying-multi-label-comments-0-9741-lb/submission_binary.csv")
dftest = pd.merge(dftest,dfpred, on="id", how="left")

#dftest = dftest.sample(1000).reset_index(drop=True)
del dfpred
gc.collect()

In [ ]:
cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
weights = [1, 2, 3, 4, 1, 5]

def generate_weighted_score(row):
    numerator = 0
    for col, wt in zip(cols, weights):
        numerator += row[col]*wt
    return numerator/sum(weights) #Why weight normalise? Because HF gives 0-1 scores/

dftest["unweighted_score"] = dftest.progress_apply(lambda row: np.mean([row[col] for col in cols]), axis=1)
dftest["weighted_score"] = dftest.progress_apply(generate_weighted_score, axis=1)

In [ ]:
dftest["weighted_score"].describe() 
dftest["bin"] = dftest["weighted_score"].round(1)#.astype(str) 
dftest["bin"] = dftest["bin"]>0.
(dftest["weighted_score"]>=0.5).value_counts()

```python
traincs = set(dftest["comment_text"].unique().tolist())
dummy = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
testcs = set(dummy["less_toxic"].unique().tolist()+dummy["more_toxic"].unique().tolist())
len(traincs), len(testcs), len(traincs-testcs), len(testcs-traincs), len(traincs.intersection(testcs))
OP: (153164, 14251, 153164, 14251, 0)
```

## Preprocessing
Should be kinda similar [to source code](https://github.com/VinAIResearch/BERTweet#preprocess).  Don't need twitter like preprocessing. But following are required:
<ol>
    <li>Replacing \n with spaces.</li>
    <li>Replacing <a href="https://en.wikipedia.org/wiki/Help:Talk_pages#Indentation">Wiki specific tags</a></li>
    <li>emoji normalizing </li>
</ol>

In [ ]:
dftest["normed_text"] = wtp_utils.preprocess_text(dftest["comment_text"])

In [ ]:
cois = ['id', 'comment_text', 'normed_text', 'unweighted_score', 'weighted_score']
dftest = dftest[cois]
dftest[cois].to_csv("cleaned_train.csv", index=False) #misnomer. This is test data with roc 0.97

# Train data

In [ ]:
from tqdm import tqdm
import gc, pandas as pd
tqdm.pandas()

traindf = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
traindf["bin"] = traindf.apply(lambda row: any([row[c]==1 for c in cols]), axis=1)
traindf = traindf.groupby("bin").sample(16225).reset_index(drop=True)

wts = pd.DataFrame([(c, traindf[c].value_counts().iloc[1]) for c in cols])
wts[1] = 1000/wts[1]
wts.set_index(0, inplace=True)
wts = wts.T.iloc[0].to_dict()

tot_wts = sum(wts.values())

traindf["new_wscore"] = traindf.progress_apply(lambda row: sum([wts[c]*row[c] for c in cols]), axis=1)
traindf["new_wscore"] = traindf["new_wscore"].apply(lambda x: min(x, .25))
traindf["new_wscore"] = (traindf["new_wscore"]-traindf["new_wscore"].min())/(traindf["new_wscore"].max()-traindf["new_wscore"].min())

cois = ["id", "comment_text", "new_wscore"]
traindf = traindf[cois]

traindf["normed_text"] = wtp_utils.preprocess_text(traindf["comment_text"])
traindf.to_csv("cleaned_train_actual.csv", index=False)

In [ ]:
from tqdm import tqdm
import gc, pandas as pd
tqdm.pandas()

traindf = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip")
cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
is_specific_t = traindf.apply(lambda row: any([row[c]==1 for c in cols[1:]]), axis=1)

# Pseudoranked train data

Obscene is usually swearwords. Insult and obscene are kinda hand-in-hand. So same league? threat and identity hate are at highest. So we have non-toxic < toxic < severe toxic < obscenity and insults < threat and identity hate


In [ ]:
#nontoxic < toxiconly < severe toxic only < (obscene/insult/both only) < (threat or identity hate or both only)
is_non_toxic = traindf["toxic"]==0
for c in cols[1:]:
    is_non_toxic &= traindf[c]==0
    
is_toxic_only = traindf["toxic"]==1
for c in ["severe_toxic", "obscene", "threat", "insult", "identity_hate"]:
    is_toxic_only &= traindf[c]==0

is_severe_toxic_only = traindf["severe_toxic"]==1
for c in ["obscene", "threat", "insult", "identity_hate"]:
    is_severe_toxic_only &= traindf[c]==0

is_oi_only = (traindf["obscene"]==1)|(traindf["insult"]==1)
for c in ["toxic", "severe_toxic", "threat", "identity_hate"]:
    is_oi_only &= traindf[c]==0

is_it_only = (traindf["threat"]==1)|(traindf["identity_hate"]==1)
for c in ["toxic", "severe_toxic", "obscene", "insult"]:
    is_it_only &= traindf[c]==0

    
traindf["rank"] = -1 #Inconclusive
traindf.loc[is_non_toxic, "rank"] = 0
traindf.loc[is_toxic_only, "rank"] = 1
traindf.loc[is_severe_toxic_only, "rank"] = 2
traindf.loc[is_oi_only, "rank"] = 3
traindf.loc[is_it_only, "rank"] = 4
traindf["rank"].value_counts()

traindf.to_csv("ranked_train_actual.csv", index=False)